

```
# This is formatted as code
```


Dhruuv agarwal 

Denoising using 2D CNN by constructing images from signal info



In [0]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import math

In [0]:
def getSNR(yRawInpTimeDomain,yPredReadyToWrite):
    minLength = min(len(yRawInpTimeDomain),len(yPredReadyToWrite))
    clippedY_True = yRawInpTimeDomain[:minLength]
    clippedY_Pred = yPredReadyToWrite[:minLength]
    signal  = np.dot(clippedY_True,clippedY_True.T)
    subtract = clippedY_True - clippedY_Pred
    noise = np.dot(subtract,subtract.T)
    
    return 10*math.log10(signal/noise)

Here we get the files as given in the document and get the required input matrices after stft and absolute to get the magnitude.

We then take the transpose of these matrices to match the format of Input matrix in the graph structure.


In [0]:
!pip install librosa # in colab, you’ll need to install this
import librosa
s, sr=librosa.load('train_clean_male.wav', sr=None)
Sdata=librosa.stft(s, n_fft=1024, hop_length=512)
sn, sr=librosa.load('train_dirty_male.wav', sr=None)
oXdata=librosa.stft(sn, n_fft=1024, hop_length=512)
Xdata = np.abs(oXdata.T)
Sdata = np.abs(Sdata.T)

snt, srt=librosa.load('test_x_01.wav', sr=None)
Xtest=librosa.stft(snt, n_fft=1024, hop_length=512)
Xtestm = np.abs(Xtest.T)

snt2, srt2=librosa.load('test_x_02.wav', sr=None)
Xtest2=librosa.stft(snt2, n_fft=1024, hop_length=512)
Xtestm2 = np.abs(Xtest2.T)

print(Xdata.shape)

(2459, 513)


In [0]:
Xinp = tf.placeholder('float',[None,20,513,1])
yinp = tf.placeholder('float',[None,513])
y = tf.placeholder('float')
is_training = tf.placeholder(tf.bool)

from sklearn.model_selection import train_test_split

#Xdata,Xval,Sdata,Sval = train_test_split(Xdata,Sdata,test_size=0.2)

In [0]:
output = []
output2 = []
trainoutput=[]
batch_size = 10

def cnn_model_fn(Xinp,yinp,mode):
  global output 
  global output2
  global batch_size
  global trainoutput

  
  input_layer = tf.reshape(Xinp, [-1, 20,513,1])

  # Convolutional Layer #1
  conv1 = tf.layers.conv2d(
      inputs=input_layer,
      filters=16,
      kernel_size=[4, 4],
      padding="same",
      activation=tf.nn.relu)
  
  # Pooling Layer #1
  pool1 = tf.layers.max_pooling2d(inputs=conv1, pool_size=[2, 2], strides=[2,2])#,data_format = 'channels_first')

  # Convolutional Layer #2 and Pooling Layer #2
  conv2 = tf.layers.conv2d(
      inputs=pool1,
      filters=32,
      kernel_size=[2, 2],
      padding="same",
      activation=tf.nn.relu
  )
  
  pool2 = tf.layers.max_pooling2d(inputs=conv2, pool_size=[2, 2], strides=[2,2])#,data_format = 'channels_first')
  pool2_flat = tf.reshape(pool2, [-1, 1 *5* 128 * 32])
  #pool2_flat= tf.layers.flatten(pool2)#,data_format= 'channels_first')

  dense = tf.layers.dense(inputs=pool2_flat, units=4048, activation=tf.nn.relu)#kernel_initializer=init,bias_initializer=init)#,data_format = 'channels_first')
  
  dropout = tf.layers.dropout(inputs=dense, rate=0.3,training=is_training)
  
  out = tf.layers.dense(inputs=dropout, units=513, activation=tf.nn.relu)#kernel_initializer=init,bias_initializer=init)
  
  mse = tf.losses.mean_squared_error(labels=yinp, predictions = out)
  
  
  # Configure the Training Op (for TRAIN mode)
  
  optimizer = tf.train.AdamOptimizer(learning_rate=0.0001).minimize(mse)
    
  if mode == 'TRAIN':
    n_epochs = 200


    with tf.Session() as sess:
      sess.run(tf.global_variables_initializer())
      imageout = Sdata[19:,:]
      valimage = imageout[2420:,:]
      imageout = imageout[:2420,:]
      
      print(imageout[0].shape,valimage.shape)
      
      images1= np.array([ np.reshape(Xdata[j:j+20], (20, 513,1)) for j in range(2440)])
      print(imageout.shape,images1.shape)
      valimages = images1[2420:]
      images = images1[:2420]
      
      testimages1= np.array([ np.reshape(Xtestm[j:j+20], (20, 513,1)) for j in range(len(Xtestm)-19)])
      testimages2= np.array([ np.reshape(Xtestm2[j:j+20], (20, 513,1)) for j in range(len(Xtestm2)-19)])
      
      indices = [j for j in range(0,2440)]
      for epoch in range(n_epochs):
        tempout=[]
        res=[]
        loss = 0
        for i in range(0,int(2420/batch_size)):   
          xbatch = images[i*batch_size:(i+1)*batch_size]
          sbatch = imageout[i*batch_size:(i+1)*batch_size,:]
          last= (i+1)*batch_size
   
          res = sess.run([optimizer,mse],feed_dict={Xinp:xbatch,yinp:sbatch,is_training:True})
          loss += res[1]
          
        xbatch = images[last:]
        sbatch = imageout[last:]
        #sbatch = np.reshape(sbatch, [1, 513])
        
        if len(xbatch) >1:
          res = sess.run([optimizer,mse],feed_dict={Xinp:xbatch,yinp:sbatch,is_training:True})
          loss += res[1]
          #tempout.append(res[2])
        tempout = sess.run(out,feed_dict = {Xinp:images1, is_training:False})
        rows = []
        for j in range(0,19):
          row = [np.random.random()/1000 for i in range(0,513)]
          rows.append(row)
        rows=np.array(rows)  
        tempout = np.concatenate((rows,tempout),axis=0)
        sht = (oXdata/Xdata.T)*tempout.T
        iSt = librosa.istft(sht, hop_length=512)
        
        print("snr:", getSNR(s,iSt))
        loss = loss/(int(2420/batch_size))
        resv = sess.run([optimizer,mse],feed_dict={Xinp:valimages,yinp:valimage,is_training:False})    
        print('Epoch',epoch,' out of ',n_epochs)#,' loss:',loss," ",resv[1])
        
      
      output = sess.run(out,feed_dict = {Xinp:testimages1, is_training:False})  
      output2=sess.run(out,feed_dict = {Xinp:testimages2, is_training:False}) 
       

In [0]:
#setparam(2,1024,513,513,100)
cnn_model_fn(Xinp,yinp,'TRAIN')
#print(output2.shape)


Instructions for updating:
Use keras.layers.conv2d instead.
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use keras.layers.max_pooling2d instead.
Instructions for updating:
Use keras.layers.dense instead.
Instructions for updating:
Use keras.layers.dropout instead.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.cast instead.
(513,) (20, 513)
(2420, 513) (2440, 20, 513, 1)
2.395818836212312
Epoch 0  out of  200  loss: 0.07954061189819125   0.019349078
3.7349354580095673
Epoch 1  out of  200  loss: 0.0587290790812674   0.018004125
5.1348441227166655
Epoch 2  out of  200  loss: 0.043505569014959   0.013986467
6.038109560187035
Epoch 3  out of  200  loss: 0.03356647283283016   0.009627054
6.680919694993499
Epoch 4  out of  200  loss: 0.027830216745562873   0.006066839
6.237181205806648
Epoch 5  out of  200  loss: 0.02491820599889928   0.

We see the SNR ends up to be 16 approx in about 200 epochs only.  

Also as we needed positive values at the end we have chosen relu activation at the last layer. 
The structure is 2 conv layer (16 filters of 4x4 and 32 filters of 2x2 ) with maxpooling after both,followed by 1 dense layer with dropout then output layer.  And the the leanrning rate was selected to be 0.0001. 

After this we get the Matrices learnt from the trained model and the Test input data, and convert them to get back the phase information as below.

This enables us to use istft and get the reconstructed version of the sound signal.

In [0]:
rows = []
for j in range(0,19):
  row = [np.random.random()/1000 for i in range(0,513)]
  rows.append(row)
rows=np.array(rows)  
final = np.concatenate((rows,output),axis=0)
shtest1 = (Xtest/Xtestm.T)*final.T
final2 = np.concatenate((rows,output2),axis=0)
shtest2 = (Xtest2/Xtestm2.T)*final2.T


In [0]:
iStftMat1 = librosa.istft(shtest1, hop_length=512)
iStftMat2 = librosa.istft(shtest2, hop_length=512)

librosa.output.write_wav('test_s_01_recons.wav', iStftMat1, srt)
librosa.output.write_wav('test_s_02_recons.wav', iStftMat2, srt2)